In [1]:
# code random forest from scratch

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [84]:
a = [0, 1, 0]
_, counts = np.unique(a, return_counts=True)
p = counts / counts.sum()
p

array([0.66666667, 0.33333333])

In [89]:
def entropy(a):
    """entropy of category distribution"""
    p = np.unique(a, return_counts=True)[1] / len(a)
    p = p[p > 0]  # make x log x = 0 for x = 0
    return -(p * np.log(p)).sum()

entropy([0, 0, 1]), entropy([0, 1, 1]), entropy([1, 1, 1])

(0.6365141682948128, 0.6365141682948128, -0.0)

In [90]:
def sigmoid(x):
    return np.exp(x) / (1 + np.exp(x))

In [186]:
def auc(label, score):
    """Assuming no ties in score."""
    label, score = np.array(label, dtype=bool), np.array(score, dtype=float)
    return (score[label] > score[~label].reshape(-1, 1)).mean()

print(auc([0, 0, 1, 1], [.2, .4, .6, .8]), 
      auc([1, 1, 0, 0], [.2, .4, .6, .8]),
      auc([0, 1, 1, 0], [.2, .4, .6, .8]))

1.0 0.0 0.5


In [187]:
def log_loss(label, score):
    label, score = np.array(label, dtype=bool), np.array(score, dtype=float)
    score = np.minimum(1 - 1e-6, np.maximum(1e-6, score))  # get rid of perfect 0s and 1
    return -1. / len(label) * (label * np.log(score) + (1 - label) * np.log(1 - score)).sum() 

print(log_loss([0, 0, 1, 1], [.2, .4, .6, .8]), 
      log_loss([1, 1, 0, 0], [.2, .4, .6, .8]),
      log_loss([0, 1, 1, 0], [.2, .4, .6, .8]))

0.3669845875401002 1.2628643221541278 0.814924454847114


In [188]:
np.random.seed(1234)
n, m = 1000, 20
X = np.random.randn(n, m)
true_coef = np.random.randn(m)
true_bias = np.random.randn()
noise = 3 * np.random.randn(n)
y = (sigmoid(X.dot(true_coef) + true_bias + noise) < .5).astype(int)
y.mean()

0.504

In [189]:
X.dot(true_coef).shape

(1000,)

In [286]:
np.random.seed(123)
frac_test = .2
test_idx = np.random.random(len(X)) < frac_test
X_train, X_test, y_train, y_test = X[~test_idx], X[test_idx], y[~test_idx], y[test_idx]

In [287]:
np.random.seed(456)
coef = np.random.random(m)
bias = np.random.randn()
y_pred = sigmoid(X_train.dot(coef) + bias)
print(log_loss(y_train, y_pred), auc(y_train, y_pred))

1.2011519872092786 0.5322850035536603


In [288]:
features = sorted(np.random.choice(m, int(np.sqrt(m)), replace=False))
features

[0, 7, 9, 14]

In [291]:
def find_split(X, y, random=False):
    X, y = np.array(X), np.array(y)
    cost = entropy(y)
    f, t, g = 0, 0, 0
    m = X.shape[1]
    features = np.random.choice(m, int(np.sqrt(m)), replace=False) if random else np.arange(m)
    for i, x in enumerate(X[:, features].T):
        # mu, sigma = x.mean(), x.std()
        thresholds = np.unique(x)
        for threshold in thresholds:
            y1, y2 = y[x < threshold], y[x >= threshold]
            if len(y1) and len(y2):
                new_cost = (len(y1) * entropy(y1) + len(y2) * entropy(y2)) / len(y)  # weighted
                gain = cost - new_cost
                if new_cost == 0:  # perfect split
                    return i, threshold, gain
                elif gain > g:
                    f = i
                    t = threshold
                    g = gain
    return f, t, g

In [292]:
np.random.seed(1212)
f, t, g = find_split(X[:, np.random.permutation(X.shape[1])], y)
idx = X[:, f] < t
idx.mean()

0.569

In [293]:
def create_tree(X, y, random=False):
    X, y = np.array(X), np.array(y)
    node = {'y_mean': y.mean(), 'n': len(X)}
    if entropy(y) > 0:
        f, t, g = find_split(X, y, random=random)
        idx = X[:, f] < t
        if idx.any() and not idx.all():
            node = {
                'feature': f,
                'threshold': t,
                'gain': g,
            }           
            node['lower'] = create_tree(X[idx], y[idx], random=random)
            node['higher'] = create_tree(X[~idx], y[~idx], random=random)
    return node

In [294]:
%%time
root = create_tree(X, y)

CPU times: user 3.22 s, sys: 5.47 ms, total: 3.22 s
Wall time: 3.23 s


In [295]:
# root

In [296]:
def predict_tree(X, node):
    if 'y_mean' in node:
        return node['y_mean'] * np.ones(len(X))
    else:
        y_pred = np.ones(len(X))
        idx = X[:, node['feature']] < node['threshold']
        y_pred[idx] = predict(X[idx], node['lower'])
        y_pred[~idx] = predict(X[~idx], node['higher'])
        return y_pred

In [297]:
%%time
y_pred = predict_tree(X, root)

CPU times: user 5.74 ms, sys: 1.27 ms, total: 7.01 ms
Wall time: 7.29 ms


In [298]:
log_loss(y, y_pred), auc(y, y_pred)

(1.0000005000290889e-06, 1.0)

but, mega-overfitted:

In [299]:
%%time
root2 = create_tree(X_train, y_train)
y_pred2 = predict_tree(X_train, root2)
print(log_loss(y_train, y_pred2), auc(y_train, y_pred2))
y_pred2_test = predict_tree(X_test, root2)
print(log_loss(y_test, y_pred2_test), auc(y_test, y_pred2_test))

1.000000500029089e-06 1.0
5.9763731121460975 0.32269503546099293
CPU times: user 2.47 s, sys: 4.23 ms, total: 2.47 s
Wall time: 2.48 s


In [324]:
def create_forest(X, y, n_trees=100):
    X, y = np.array(X), np.array(y)
    trees = []
    for _ in range(n_trees):
        idx = np.random.choice(len(X), len(X), replace=True)  # bootstrapping
        trees.append(create_tree(X[idx], y[idx], random=True))
    return trees

In [325]:
def predict_forest(X, trees):
    return np.array([predict_tree(X, tree) for tree in forest]).mean(axis=0)

In [328]:
%%time
forest = create_forest(X_train, y_train, n_trees=30)

CPU times: user 7.06 s, sys: 5.78 ms, total: 7.06 s
Wall time: 7.09 s


In [329]:
y_pred_f = predict_forest(X_train, forest)
print(log_loss(y_train, y_pred_f), auc(y_train, y_pred_f))
y_pred_f_test = predict_forest(X_test, forest)
print(log_loss(y_test, y_pred_f_test), auc(y_test, y_pred_f_test))

0.6051876722365546 0.7966358682776593
0.6503269007820847 0.7094731509625126


In [307]:
y_pred_f = predict_forest(X_train, forest)
print(log_loss(y_train, y_pred_f), auc(y_train, y_pred_f))
y_pred_f_test = predict_forest(X_test, forest)
print(log_loss(y_test, y_pred_f_test), auc(y_test, y_pred_f_test))

0.6071706507420548 0.7976072020848141
0.6583788589033591 0.6742654508611955
